# Create scheduler pickles for testing `schedview`

In [ ]:
from collections import namedtuple
import lzma
import pickle
from pathlib import Path
import warnings

In [ ]:
import os
import numpy as np
import pandas as pd
import healpy as hp
import rubin_scheduler
from astropy.time import Time
import astropy.units as u
import astropy.coordinates
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_scheduler.scheduler.surveys.field_survey import FieldSurvey
from rubin_scheduler.scheduler.schedulers import CoreScheduler
import rubin_scheduler.scheduler.basis_functions as bf
from rubin_scheduler.scheduler import sim_runner
import schedview.collect
import schedview.compute
from rubin_scheduler.utils import survey_start_mjd

In [ ]:
SchedulerPickleContent = namedtuple('SchedulerPickleContent', ['scheduler', 'conditions'])

In [ ]:
mjd_start = survey_start_mjd()
model_observatory = ModelObservatory(mjd_start=mjd_start)
nside = model_observatory.nside
sim_duration = (4*u.hour).to(u.day).value

# Create a near-baseline sample scheduler

Use the `make_sample_test_data.py` script in `schedview`, found in `util/sample_data`:

# Get a production auxtel scheduler

Use python `efd_sched.py` script in `schedview`, found in `util`. For example:

Note that the date specified is the date the entry was added to the EFD, rather than the date of the pickle itself, so there is some offset. For example, the above command returns the file
`auxtel_scheduler_2023-10-11T043052.526.p`.

You can use `efd_sched.py` in this command to query for which EFD date corresponds to a given file:
```
bash$ python util/efd_sched.py query '2023-10-10' | grep '2023-10-11T04.30.52'
55	2023-10-11 04:30:16.625007+00:00	https://s3.cp.lsst.org/rubinobs-lfa-cp/Scheduler:2/Scheduler:2/2023/10/10/Scheduler:2_Scheduler:2_2023-10-11T04:30:52.526.p
```

# Create schedulers with problems

## Introduction

I start by making an example scheduler with two tiers. The first tier is an equatorial survey in with fields every hour, such that there should be fields visible at all times.
The second tier is a single greed survey (in g) covering the whole sky.

## Make some basis functions

In [ ]:
def make_sky_bf_list(band='g', nside=32):
    not_twilight = bf.feasibility_funcs.NotTwilightBasisFunction()
    moon_limit = bf.mask_basis_funcs.MoonAvoidanceBasisFunction(nside=nside, moon_distance=30.0)
    zenith_limit = bf.mask_basis_funcs.ZenithShadowMaskBasisFunction(nside=nside, min_alt=20.0, max_alt=82.0)
    sky_brightness_limit = bf.basis_functions.SkybrightnessLimitBasisFunction(nside=nside, filtername=band, sbmin=18.5, sbmax=30)
    wind_limit = bf.basis_functions.AvoidDirectWind(5)
    m5diff = bf.basis_functions.M5DiffBasisFunction(filtername=band, nside=nside)
    basis_functions = [
        not_twilight,
        moon_limit,
        zenith_limit,
        sky_brightness_limit,
        wind_limit,
        m5diff
    ]
    return basis_functions

In [ ]:
def make_field_bf_list(ra, decl, band='g', nside=32):
    basis_functions = make_sky_bf_list(band=band, nside=nside)
    basis_functions.append(bf.feasibility_funcs.HourAngleLimitBasisFunction(RA=ra, ha_limits=[[22,24], [0,2]]))
    return basis_functions

In [ ]:
def make_field_survey(ra, decl, band='g', nside=32):
    basis_functions = make_field_bf_list(ra, decl, band=band, nside=nside)
    sequence = band
    nvis = [1]*len(band)
    survey_name = f"field_{ra}_{'n' if decl<0 else 'p'}{np.abs(decl)}_{band}"
    survey = FieldSurvey(basis_functions, np.array([ra]), np.array([decl]), sequence=sequence, nvis=nvis, nside=nside, survey_name=survey_name, reward_value=1.0)
    return survey

## Make an equatorial survey in g

Start by making a list of field surveys in g on the equator, with fields spaced every hour (15 degrees):

In [ ]:
decl = 0
band = 'g'
field_surveys = [make_field_survey(ra, decl, band, nside) for ra in range(0, 360, 15)]

As a fallback, greate a greedy survey in g that covers the whole sky:

In [ ]:
sky_basis_functions = make_sky_bf_list(band=band, nside=nside)
weights = [1] * len(sky_basis_functions)
greedy_surveys = [rubin_scheduler.scheduler.surveys.surveys.GreedySurvey(sky_basis_functions, weights, filtername=band, survey_name=f"greedy_{band}")]

Actually create the scheduler:

In [ ]:
scheduler = CoreScheduler([field_surveys, greedy_surveys], nside=nside)

Configure the scheduler to keep reward values:

In [ ]:
scheduler.keep_rewards = True

Actually run the survey for a little bit:

In [ ]:
observatory, scheduler, observations, reward_df, obs_rewards = sim_runner(
    model_observatory,
    scheduler,
    sim_start_mjd=mjd_start,
    sim_duration=sim_duration,
    record_rewards=True,
)
conditions = scheduler.conditions

Save the survey:

In [ ]:
output_tuple = (scheduler, conditions)
if not os.path.isdir('../tmp'):
    os.mkdir('../tmp')
fname = Path('../tmp/eq_field_survey_v0.p.xz').resolve()
with lzma.open(fname, 'wb') as sched_out:
    pickle.dump(output_tuple, sched_out)

fname

In [ ]:
print(f"Conditions calculated for {Time(conditions.mjd[0], format='mjd').iso} (mjd {conditions.mjd[0]})")

Take a look at one survey:

In [ ]:
scheduler_reward_df = scheduler.make_reward_df(conditions, accum=True)
scheduler_reward_df.loc[(0,1),:]

Take a look at the rewards for all surveys.

In [ ]:
scheduler_summary_df = schedview.compute.make_scheduler_summary_df(scheduler, conditions, scheduler_reward_df)
scheduler_summary_df.query('tier=="tier 0"')

# Make a survey where everything is infeasible

Create a combination of survey and conditions where everything is infeasible, blocked either by wind or by sky brightess, but where there are observations not blocked by both.

The first full moon after the nominal survey start is 2025-05-12. This should be in the default limited sky brightness data set.

Find the next full moon after the start of the survey:

In [ ]:
month_mjds = np.arange(int(mjd_start), int(mjd_start)+30)
month_phases = np.array([model_observatory.almanac.get_sun_moon_positions(mjd)['moon_phase'] for mjd in month_mjds])
mjd = month_mjds[np.argmax(month_phases)]
Time(mjd, format='mjd').iso

Find a time during the night where the moon is at a zd of about 55 degrees.

In [ ]:
sunset_info = model_observatory.almanac.get_sunset_info(mjd)
sunset, sunrise = sunset_info[3], sunset_info[4]
for mjd in np.arange(sunset, sunrise, 1.0/(24*4)):
    moon_alt = model_observatory.almanac.get_sun_moon_positions(mjd)['moon_alt']
    if np.abs(np.degrees(moon_alt)-55) < 5:
        break

time_to_sched = Time(mjd, format='mjd')
time_to_sched.iso, mjd

In [ ]:
model_observatory.mjd = time_to_sched.mjd

Set the wind so it comes from the opposite direction.

In [ ]:
moon_az = model_observatory.almanac.get_sun_moon_positions(time_to_sched.mjd)['moon_az']
wind_az = moon_az - np.pi ;# opposite moon
wind_data = rubin_scheduler.site_models.ConstantWindData(wind_speed=18.0, wind_direction=wind_az)
model_observatory.wind_data = wind_data

Actually build our instance of Conditions:

In [ ]:
conditions = model_observatory.return_conditions()

In [ ]:
scheduler.make_reward_df(conditions, accum=True)

In [ ]:
schedview.compute.make_scheduler_summary_df(scheduler, conditions)

In [ ]:
output_tuple = (scheduler, conditions)
fname = Path('../tmp/eq_field_survey_v0_infeasible1.p.xz').resolve()
with lzma.open(fname, 'wb') as sched_out:
    pickle.dump(output_tuple, sched_out)

fname

# Making pickles available

Copy them to `/sdf/group/rubin/web_data/sim-data/sched_pickles` at the USDF for them to be visible at https://s3df.slac.stanford.edu/data/rubin/sim-data/sched_pickles